<a href="https://colab.research.google.com/github/kaartikaykhanduri/vehicle-damage-detection/blob/main/Cardetetction_location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!unzip "/content/gdrive/MyDrive/car-damage-dataset.zip"

Archive:  /content/gdrive/MyDrive/car-damage-dataset.zip
replace car-damage-dataset/data1/training/damage/0001.JPEG? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
import urllib.request
import h5py
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from IPython.display import Image, display, clear_output
from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.models import Sequential, load_model, Model
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Activation, Dropout, Flatten, Dense, Input
from keras.regularizers import l2, l1
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array,load_img
from keras.callbacks import ModelCheckpoint, History
from keras import backend as K
from keras.utils.data_utils import get_file

In [ ]:
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1./255)

    model = VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(train_data_dir, target_size=(img_width,img_height), batch_size=batch_size, class_mode=None, shuffle=False)
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples//batch_size)
    np.save(location+'/bottleneck_features_train.npy', bottleneck_features_train)

    generator = datagen.flow_from_directory(validation_data_dir, target_size=(img_width,img_height), batch_size=batch_size, class_mode=None, shuffle=False)
    bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples//batch_size)
    np.save(location+'/bottleneck_features_validation.npy', bottleneck_features_validation)

In [ ]:
def print_best_model_results(model_hist):
    best_epoch = np.argmax(model_hist['val_accuracy'])
    print('epoch:', best_epoch+1, ', val_accuracy:', model_hist['val_accuracy'][best_epoch], ', val_loss:', model_hist['val_loss'][best_epoch])


In [ ]:
def plot_metrics(hist, stop=50):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,4))
    axes = axes.flatten()

    axes[0].plot(range(stop), hist['accuracy'], label='Training')
    axes[0].plot(range(stop), hist['val_accuracy'], label='Validation')
    axes[0].set_title('Accuracy')
    axes[0].set_ylabel('Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].legend(loc='lower right')

    axes[1].plot(range(stop), hist['loss'], label='Training')
    axes[1].plot(range(stop), hist['val_loss'], label='Validation')
    axes[1].set_title('Loss')
    axes[1].set_ylabel('Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].legend(loc='upper right')

    plt.tight_layout();

    print("Best Model:")
    print_best_model_results(hist)

In [ ]:
def train_categorical_model():
    train_data = np.load(location+'/bottleneck_features_train.npy')
    train_labels = np.array([0]*(416) + [1]*(288) + [2]*(272))
    train_labels = to_categorical(train_labels)

    validation_data = np.load(location+'/bottleneck_features_validation.npy')
    validation_labels = np.array([0]*(73) + [1]*(53) + [2]*(50))
    validation_labels = to_categorical(validation_labels)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    checkpoint = ModelCheckpoint(top_model_weights_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True,mode='auto')

    fit = model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(validation_data,validation_labels), callbacks=[checkpoint])

    with open(location+'/top_history.txt', 'w') as f:
        json.dump(fit.history, f)

    return model,fit.history

In [ ]:
def finetune_categorical_model():
    input_tensor = Input(shape=(256,256,3))
    base_model = VGG16(weights='imagenet',include_top= False,input_tensor=input_tensor)
    print("Model loaded.")
    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(3, activation='softmax'))

    top_model.load_weights(top_model_weights_path)

    model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

    for layer in model.layers[:25]:
        layer.trainable = False

    model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.00001, momentum=0.9), metrics=['accuracy'])

    train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(validation_data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')

    checkpoint = ModelCheckpoint(fine_tuned_model_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')

    fit = model.fit(train_generator, steps_per_epoch=nb_train_samples//batch_size, epochs=epochs, validation_data=validation_generator, validation_steps=nb_validation_samples//batch_size, verbose=1, callbacks=[checkpoint])

    with open(location+'/ft_history.txt', 'w') as f:
        json.dump(fit.history, f)

    return model, fit.history

In [ ]:
def evaluate_categorical_model(model, directory, labels):
    datagen = ImageDataGenerator(rescale=1./255)

    generator = datagen.flow_from_directory(directory, target_size=(img_height,img_width), batch_size=batch_size, class_mode='categorical', shuffle=False)

    predictions = model.predict_generator(generator, len(labels))

    pred_labels = [0 if i<0.5 else 1 for i in predictions]

    print('')
    print(classification_report(validation_labels, pred_labels))
    print('')
    cm = confusion_matrix(validation_labels, pred_labels)
    return cm

LOCATION OF THE DAMAGE ( FRONT , SIDE , REAR)

In [ ]:
location = 'car-damage-dataset/data2'
top_model_weights_path = location+'/my_model_weights.h5'
fine_tuned_model_path = location+'/ft_model.h5'
model1 = location+'/bottleneck_fc_model.h5'
train_data_dir = location+'/training'
validation_data_dir = location+'/validation'
train_samples = [len(os.listdir(train_data_dir+'/'+i)) for i in sorted(os.listdir(train_data_dir))]
nb_train_samples = 976
validation_samples = [len(os.listdir(validation_data_dir+'/'+i)) for i in sorted(os.listdir(validation_data_dir))]
nb_validation_samples = 176

img_width, img_height = 256,256
epochs = 1
batch_size = 16

In [ ]:
save_bottleneck_features()

Found 985 images belonging to 3 classes.


<ipython-input-18-670539d0b709>:7: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  bottleneck_features_train = model.predict_generator(generator, nb_train_samples//batch_size)


Found 179 images belonging to 3 classes.


<ipython-input-18-670539d0b709>:11: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples//batch_size)


In [ ]:
d3_model, d3_history = train_categorical_model()

61/61 [==============================] - ETA: 0s - loss: 3.9601 - accuracy: 0.4191
Epoch 1: val_accuracy improved from -inf to 0.48864, saving model to car-damage-dataset/data2/my_model_weights.h5
61/61 [==============================] - 8s 118ms/step - loss: 3.9601 - accuracy: 0.4191 - val_loss: 1.0463 - val_accuracy: 0.4886


In [ ]:
d3_model.save('my_model_weights.h5')
d3_model.save('ft_model.h5')

In [ ]:
ft_model, ft_history = finetune_categorical_model()

Model loaded.
Found 985 images belonging to 3 classes.
Found 179 images belonging to 3 classes.


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


61/61 [==============================] - ETA: 0s - loss: 1.0502 - accuracy: 0.4892 
Epoch 1: val_accuracy improved from -inf to 0.48295, saving model to car-damage-dataset/data2/ft_model.h5
61/61 [==============================] - 858s 14s/step - loss: 1.0502 - accuracy: 0.4892 - val_loss: 1.0522 - val_accuracy: 0.4830


In [ ]:
def pipelocation(image_path, model):
    urllib.request.urlretrieve(image_path, 'save.jpg')
    img = load_img('save.jpg', target_size=(256,256))
    x = img_to_array(img)
    x = x.reshape((1,)+x.shape)/255
    pred = model.predict(x)
    pred_labels = np.argmax(pred, axis=1)
    d = {0:'Front', 1:'Rear', 2:'Side'}
    for key in d.keys():
        if pred_labels[0] == key:
            print("Validating location of damage....Result:",d[key])
            return d[key]
    print("Severity assessment complete.")


In [ ]:
pipelocation("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT4DEd3yM1SERRZkPl25MZGIIxtl127-R8IrQ&usqp=CAU",ft_model)

1/1 [==============================] - 1s 818ms/step
Validating location of damage....Result: Front


'Front'

HOW MUCH DAMAGE (MINOR , MODERATE , SEVERE)

In [ ]:
location = 'car-damage-dataset/data3'
top_model_weights_path = location+'/my_model_weights.h5'
fine_tuned_model_path = location+'/ft_model.h5'
model1 = location+'/bottleneck_fc_model.h5'
train_data_dir = location+'/training'
validation_data_dir = location+'/validation'
train_samples = [len(os.listdir(train_data_dir+'/'+i)) for i in sorted(os.listdir(train_data_dir))]
nb_train_samples = 976
validation_samples = [len(os.listdir(validation_data_dir+'/'+i)) for i in sorted(os.listdir(validation_data_dir))]
nb_validation_samples = 176

img_width, img_height = 256,256
epochs = 1
batch_size = 16

In [ ]:
save_bottleneck_features()

Found 979 images belonging to 3 classes.


<ipython-input-18-670539d0b709>:7: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  bottleneck_features_train = model.predict_generator(generator, nb_train_samples//batch_size)


Found 176 images belonging to 3 classes.


<ipython-input-18-670539d0b709>:11: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples//batch_size)


In [ ]:
d2_model, d2_history = train_categorical_model()

61/61 [==============================] - ETA: 0s - loss: 4.1823 - accuracy: 0.4580
Epoch 1: val_accuracy improved from -inf to 0.47727, saving model to car-damage-dataset/data3/my_model_weights.h5
61/61 [==============================] - 7s 106ms/step - loss: 4.1823 - accuracy: 0.4580 - val_loss: 0.9832 - val_accuracy: 0.4773


In [ ]:
d2_model.save('my_model_weights.h5')
d2_model.save('ft_model.h5')

In [ ]:
ft2_model, ft2_history = finetune_categorical_model()

Model loaded.
Found 979 images belonging to 3 classes.
Found 176 images belonging to 3 classes.
61/61 [==============================] - ETA: 0s - loss: 1.1147 - accuracy: 0.3586 
Epoch 1: val_accuracy improved from -inf to 0.31818, saving model to car-damage-dataset/data3/ft_model.h5
61/61 [==============================] - 864s 14s/step - loss: 1.1147 - accuracy: 0.3586 - val_loss: 1.1008 - val_accuracy: 0.3182


In [ ]:
def pipedquality(image_path, model):
    urllib.request.urlretrieve(image_path, 'save.jpg')
    img = load_img('save.jpg', target_size=(256,256))
    x = img_to_array(img)
    x = x.reshape((1,)+x.shape)/255
    pred = model.predict(x)
    pred_labels = np.argmax(pred, axis=1)
    d = {0:'minor', 1:'moderate', 2:'Severe'}
    for key in d.keys():
        if pred_labels[0] == key:
            print("Validating damage quantity....Result:",d[key])
    print("Damage assessment complete.")

INPUT IMAGE

In [ ]:
pipedquality("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQJCbs1451TX7QnB9bGmrmRZGi0teOB9ASREQ&usqp=CAU",ft2_model)

1/1 [==============================] - 1s 796ms/step
Validating damage quantity....Result: minor
Damage assessment complete.
